In [1]:

# Importing essential libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Selecting 10 liquid stocks from different sectors (NSE)
tickers = [
    'INFY.NS', 'TCS.NS', 'RELIANCE.NS', 'HDFCBANK.NS',
    'ITC.NS', 'LT.NS', 'HINDUNILVR.NS', 'SBIN.NS',
    'BHARTIARTL.NS', 'MARUTI.NS'
]

# Print tickers for confirmation
print("Selected tickers:", tickers)

In [ ]:
# Downloading 10 years of historical daily data correctly for adjusted close prices
data = yf.download(tickers, start='2015-01-01', end='2025-01-01', auto_adjust=True)

# 'auto_adjust=True' means downloaded data is already adjusted for splits/dividends, so we can directly use the 'Close' column and the result will already be a DataFrame of Close prices

# Displaying the first few rows
data.head()


In [ ]:
# Computing returns
avg_returns = ((data["Close"] - data.shift(1)["Close"])/data.shift(1)["Close"]).dropna()

# Displaying first few rows of returns
avg_returns.head()

In [ ]:
# Defining periods
in_sample = avg_returns.loc['2015-01-01':'2020-12-31']
out_sample = avg_returns.loc['2021-01-01':'2022-12-31']

# Printing shapes to confirm
print("In-sample data shape:", in_sample.shape)
print("Out-of-sample data shape:", out_sample.shape)

# Checking the time ranges
print("\nIn-sample range:", in_sample.index.min(), "to", in_sample.index.max())
print("Out-of-sample range:", out_sample.index.min(), "to", out_sample.index.max())

In [ ]:
# Mean vector (daily)
mean_daily = in_sample.mean()

# Covariance matrix (daily)
cov_daily = in_sample.cov()
print("cov_daily", cov_daily.shape)
print(cov_daily)

# Correlation matrix
corr_matrix = in_sample.corr()

#  Annualizing (because these are daily log‑returns)
#  Convention: 252 trading days per year
mean_annual = mean_daily * 252
cov_annual = cov_daily * 252
std_annual = np.sqrt(np.diag(cov_annual))

print(" Mean return vector (annualized):")
display(mean_annual.to_frame("Expected Annual Return"))

print("\n Covariance matrix (annualized):")
display(cov_annual)

print("\n Correlation matrix:")
display(corr_matrix)

In [ ]:
%pip install cvxpy
import cvxpy as cp
import itertools

tickers_list = list(cov_annual.columns)
results = []

for k in range(1, len(tickers_list) + 1):
    subset_vars = []
    combs = list(itertools.combinations(tickers_list, k))
    print(f"Computing {len(combs)} combinations for k = {k} ...")
    
    for subset in combs:
        var_p = portfolio_variance(cov_annual_clean, subset)
        subset_vars.append(var_p)